# Create CMAQ-Ready File from Shapefile

This Notebook uses geopandas and cmaqsatproc to create IOAPI-like files for CMAQ. Geopandas supports STRtree optimized searches (via pygeos) and projection conversions. This simplifies the process to basic steps:

1. Process shapefile
    * Read in native projection.
    * Filter for in CMAQ domain.
    * Optionally, custom extra processing.
2. Calculate area overlap.
    * Perform grid cell intersections with shapefile polygons.
    * Optionally, define a weighted value (e.g, fraction of population).
    * Aggregate results to grid cell level.
    * Find largest area contributor.
    * Calculate total cell overlap.
3. Output
    * Store results as variables.
    * Save as IOAPI-like file



In [ ]:
# Install Libraries, only needs to be run first time on each machine
#!python -m pip install -qq cmaqsatproc geopandas xarray netcdf4 pycno

In [ ]:
# Download an example shapefile if you don't already have one.
#!wget https://www2.census.gov/geo/tiger/GENZ2022/shp/cb_2022_us_state_500k.zip # use with STUSPS
#!wget http://naturalearth.s3.amazonaws.com/110m_cultural/ne_110m_admin_0_countries.zip # use with ADM0_A3
#!wget https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/10m/cultural/ne_10m_admin_1_states_provinces.zip # us with iso_3166_2

In [ ]:
import shp2cmaq
import warnings
# ignore warnings
warnings.simplefilter('ignore')

## Set Configuration

In [ ]:
# shppath : str
#     Path to a shapefile or zip file containing a shapefile
shppath = 'cb_2022_us_state_500k.zip'

# attrkey : str
#     Column to group shapes by e.g., STUSPS of census (AL, NC, etc)
attrkey = 'STUSPS'

# gdnam : str
#     Name of grid definition within gdpath (e.g., 12US1, 108NHEMI2)
gdnam = '12US1'
gdpath = None # None uses built-in; or specify your own GRIDDESC path

## Run Processor

In [ ]:
outpath = shp2cmaq.shp2cmaq(shppath, attrkey, gdnam, gdpath=None, verbose=1)

## Plot Result

In [ ]:
import matplotlib.pyplot as plt
import cmaqsatproc as csp


igf = csp.open_ioapi(outpath)
domkey = [k for k in list(igf.data_vars) if k.endswith('DOM')][0]
totkey = [k for k in list(igf.data_vars) if k.endswith('TOT')][0]

fig, axx = plt.subplots(1, 2, figsize=(12, 4))
igf[domkey].where(lambda x: x > -999).plot(ax=axx[0], cmap='nipy_spectral')
igf[totkey].plot(ax=axx[1], cmap='YlOrRd')
_ = igf.csp.cno.drawcountries(ax=axx)
name2idx = eval(igf[domkey].description)
print('Dominant Index')
print(str({v: k for k, v in name2idx.items()}))